## Imports

In [32]:
import argparse
import os
import time
import sys
import random
import gc
import json
import pickle
from collections import Counter, namedtuple
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm
from scipy.linalg import eigh

from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_validate,
    TimeSeriesSplit,
    KFold,
    StratifiedKFold,
)
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import (
    make_scorer,
    confusion_matrix,
    ConfusionMatrixDisplay,
    balanced_accuracy_score,
    precision_recall_fscore_support as score,
    f1_score,
    r2_score,
    mean_absolute_error as mae,
    mean_squared_error as mse
)
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.cross_decomposition import CCA
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier

from pyriemann.estimation import XdawnCovariances, BlockCovariances, Covariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM, TSclassifier
from pyriemann.utils.mean import mean_riemann

from causallearn.search.FCMBased import lingam
from tigramite import data_processing as dp, independence_tests, pcmci

from IPython.display import display
from tqdm.notebook import tqdm

manualSeed = 111
SEED = manualSeed
FREQ = 250
random.seed(manualSeed)

## Read data

Functions to upload subjects data

In [4]:
def zscore(arr, mean=None, std=None):
    # arr.shape = (K_EVENTS, K_CHANNELS, TIMESTEPS)
    arr_mean = arr.mean(axis=(0, 2), keepdims=True) if mean is None else mean.reshape(1, -1, 1)
    arr_std = arr.std(axis=(0, 2), keepdims=True) if std is None else std.reshape(1, -1, 1)
    return (arr - arr_mean) / arr_std


def load_subject(npz_path: Path, **kwargs):
    d = np.load(npz_path, allow_pickle=True)
    # signals.shape = (K_EVENTS, EEG_CHANNELS + IMU_CHANNELS, TIMESTEPS)
    signals = d['signals'].astype(np.float32)
    channels = d['channels']
    target = d['target']

    # ----- split scalp EEG vs noise vs "other" -----
    # Assumption: first block = scalp, second block = noise (same length)
    idx_noise_start = np.where(np.char.startswith(channels, 'N-'))[0][0]
    scalp = signals[:, :idx_noise_start, :]
    noise = signals[:, idx_noise_start:2*idx_noise_start, :]
    other = signals[:, 2*idx_noise_start:, :]
    eeg = scalp - noise

    imu_names = (
        'LISCM', 'LSSCM', 'LSTrap', 'LITrap',
        'RITrap', 'RISCM', 'RSSCM', 'RSTrap',
        'Participant_Paddle_Acc_X(g)',
        'Participant_Paddle_Acc_Y(g)',
        'Participant_Paddle_Acc_Z(g)'
    )
    imu_idx = np.array(
        [int(np.where(channels == n)[0][0]) for n in imu_names]
    )
    imu = signals[:, imu_idx, :]
    imu_idx -= 2*idx_noise_start

    eeg_mean, eeg_std = kwargs.get('eeg_mean'), kwargs.get('eeg_std')
    imu_mean, imu_std = kwargs.get('imu_mean'), kwargs.get('imu_std')
    imu_mean = imu_mean[imu_idx] if imu_mean is not None else None
    imu_std = imu_std[imu_idx] if imu_std is not None else None
    
    eeg = zscore(eeg, eeg_mean, eeg_std)
    imu = zscore(imu, imu_mean, imu_std)

    target[target == 2] = 1

    return eeg, imu, target

Extract pre-calculated mean and std from `collect_data_*.ipynb`

In [7]:
PATH_HUMAN = "../data/human-player/signals-stats/"

eeg_mean = np.load(os.path.join(PATH_HUMAN, "eeg_mean.npy"))
eeg_std = np.load(os.path.join(PATH_HUMAN, "eeg_std.npy"))
imu_mean = np.load(os.path.join(PATH_HUMAN, "imu_mean.npy"))
imu_std = np.load(os.path.join(PATH_HUMAN, "imu_std.npy"))

Data pipeline

In [8]:
root = Path("../data/human-player/data/")
eeg_list, imu_list, y_list, subjects_list = [], [], [], []

for npz_file in tqdm(sorted(root.glob("signals_*.npz"))):
    subject_id = int(npz_file.name[-6:-4])
    eeg, imu, target = load_subject(
        npz_file,
        eeg_mean=eeg_mean,
        eeg_std=eeg_std,
        imu_mean=imu_mean,
        imu_std=imu_std
    )
    assert len(eeg) == len(imu) == len(target)
    assert eeg.shape[2] == imu.shape[2]
    assert eeg.shape[1] == 119
    assert 3 <= imu.shape[1] <= 23
    
    eeg_list.append(eeg)
    imu_list.append(imu)
    y_list.append(target)
    subjects_list.append([subject_id] * len(target))

eeg_total = np.concatenate(eeg_list, axis=0)         # (N, K_EEG, K_TS)
imu_total = np.concatenate(imu_list, axis=0)         # (N, K_IMU, K_TS)
y_total = np.concatenate(y_list, axis=0).astype(int) # (N,)
subjects_total = np.concatenate(subjects_list, axis=0)

del eeg_list, imu_list, y_list

print("Loaded:", eeg_total.shape, imu_total.shape, y_total.shape)

  0%|          | 0/25 [00:00<?, ?it/s]

Loaded: (18826, 119, 250) (18826, 11, 250) (18826,)


## Riemannian space

| Method | 200 EEG Matrices | 1000 EEG Matrices |
| ------ | ---------------- | ----------------- |
| SCM    | 1.22             | 22.4              |
| lwf    | 1.05             | 7.4               |
| oas    | 1.21             | 10.6              |
| mcd    | 22.8             | > 1 minute        |
| hub    | 8.21             | > 1 minute        |

We'll stop on `oas` method

Scikit-lean materials: [link](https://scikit-learn.org/stable/modules/covariance.html#shrunk-covariance)

pyRiemann materials: [link](https://pyriemann.readthedocs.io/en/latest/auto_examples/covariance-estimation/plot_covariance_estimation_robust.html#sphx-glr-auto-examples-covariance-estimation-plot-covariance-estimation-robust-py)

**Code**:
```python
%%time
eeg_slice = eeg_total[:1000]
y_slice = y_total[:1000]

xd  = XdawnCovariances(nfilter=8, estimator='scm').fit(eeg_slice, y_slice)
cov = xd.transform(eeg_slice)                           # (N, P, P)  P=16
ts  = TangentSpace(metric='riemann').fit(cov)
eeg_feat = ts.transform(cov)    
```

In [150]:
%%time
def imu_stats(arr):
    # arr: (N, C, T) → output shape (N, (C+1)*4)
    acceleration = np.sqrt(np.sum(np.square(arr[:, -3:, :]), axis=1, keepdims=True))
    arr_stacked = np.concatenate((arr, acceleration), axis=1)
    mean = arr_stacked.mean(axis=2)
    std = arr_stacked.std(axis=2)
    mx = arr_stacked.max(axis=2)
    mn = arr_stacked.min(axis=2)
    return np.concatenate([mean, std, mx, mn], axis=1)


xd  = XdawnCovariances(nfilter=8, estimator='oas').fit(eeg_total, y_total)
cov = xd.transform(eeg_total)                           # (N, P, P)  P=16
ts  = TangentSpace(metric='riemann').fit(cov)
eeg_feat = ts.transform(cov)                         # (N, P*(P+1)/2)

imu_feat = imu_stats(imu_total) # (N, (C+1)*4)

X_feat = np.hstack([eeg_feat, imu_feat])             # (N, D)

CPU times: user 1min 49s, sys: 1min 52s, total: 3min 42s
Wall time: 1min 18s


In [151]:
eeg_feat.shape, imu_feat.shape, X_feat.shape

((18826, 1176), (18826, 48), (18826, 1224))

In [ ]:
del eeg_feat, imu_feat, X_feat

## Baseline

### No models

Target distribution

In [122]:
labels, counts = np.unique(y_total, return_counts=True)
dict(zip(labels, counts / counts.sum()))

{-1: 0.09768405396791671, 0: 0.8405927971953681, 1: 0.06172314883671518}

Predict only zeros

In [127]:
def get_scores(y_true, y_pred):
    return {
        'macro': round(f1_score(y_true, y_pred, average='macro'), 3),
        'weighted': round(f1_score(y_true, y_pred, average='weighted'), 3),
    }

In [128]:
get_scores(y_total, np.zeros_like(y_total))

{'macro': 0.304, 'weighted': 0.768}

Random prediction (with equal chances)

In [132]:
np.random.seed(SEED)
rand_pred_score = np.zeros(2)
N = 100

for _ in range(N):
    rand_prediction = np.random.randint(low=-1, high=2, size=len(y_total))
    rand_pred_score += np.array(
        list(get_scores(y_total, rand_prediction).values())
    )
rand_pred_score /= N
dict(zip(('macro', 'weighted'), rand_pred_score))

{'macro': 0.24454999999999988, 'weighted': 0.42286999999999986}

Random prediction (proportional chances)

In [139]:
def chances_sampler(values, probs, size):
    assert np.allclose(np.sum(probs), 1)
    
    unif_sample = np.random.rand(size)
    ranges = [0] + np.cumsum(probs).tolist()
    samples = np.full(size, values[-1])

    for ind, value in enumerate(values):
        mask = np.bitwise_and(ranges[ind] <= unif_sample, unif_sample < ranges[ind + 1])
        samples[mask] = value

    return samples


np.random.seed(SEED)
rand_pred_score = np.zeros(2)
N = 100

for _ in range(N):
    rand_prediction = chances_sampler([-1, 0, 1], [0.1, 0.84, 0.06], y_total.size)
    rand_pred_score += np.array(
        list(get_scores(y_total, rand_prediction).values())
    )

rand_pred_score /= N
dict(zip(('macro', 'weighted'), rand_pred_score))

{'macro': 0.33304, 'weighted': 0.7197199999999997}

**Summary**

|             | only "neutral" | random (equal) | random (prop) |
|-------------|----------------|----------------|---------------|
| f1-macro    |      0.304     |      0.244     |   **0.333**   |
| f1-weighted |    **0.768**   |      0.423     |     0.720     |

### Cross-validation

In [53]:
%%time
MODEL_NAMES = ("LogReg", "kNN", "CatBoost")


def instantiate_model(model_nm):
    assert model_nm in MODEL_NAMES

    if model_nm == "LogReg":
        return make_pipeline(
            StandardScaler(),
            LogisticRegression(
                max_iter=2000,
                class_weight='balanced'
            )
        )
    elif model_nm == "kNN":
        return make_pipeline(
            StandardScaler(),
            KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        )
    elif model_nm == "CatBoost":
        return CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    else:
        return None


datasets = {
    'EEG_ONLY': eeg_feat,
    'IMU_ONLY': imu_feat,
    'TOTAL': X_feat
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
scoring = {
    'f1-macro': make_scorer(f1_score, average='macro'),
    'f1-weighted': make_scorer(f1_score, average='weighted')
}

baseline_results_cv = []

for dataset_nm, dataset in tqdm(datasets.items()):
    for model_nm in tqdm(MODEL_NAMES):
        model = instantiate_model(model_nm)
        scores = cross_validate(model, dataset, y_total, cv=cv, scoring=scoring, n_jobs=4)
        row = {
            'model': model_nm,
            'data': dataset_nm,
            'f1_macro_mean': scores['test_f1-macro'].mean(),
            'f1_macro_std': scores['test_f1-macro'].std(),
            'f1_weighted_mean': scores['test_f1-weighted'].mean(),
            'f1_weighted_std': scores['test_f1-weighted'].std(),
        }
        baseline_results_cv.append(row)
    gc.collect()
        
baseline_results_cv = pd.DataFrame(baseline_results_cv)
baseline_results_cv.to_csv('../experiment-results/human-player/baseline_cv.csv', index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 2.97 s, sys: 2.11 s, total: 5.07 s
Wall time: 5min 51s


In [57]:
baseline_results_cv

,model,data,f1_macro_mean,f1_macro_std,f1_weighted_mean,f1_weighted_std
0,LogReg,EEG_ONLY,0.384565,0.008038,0.627462,0.003938
1,kNN,EEG_ONLY,0.340214,0.007964,0.768822,0.003023
2,CatBoost,EEG_ONLY,0.304465,0.000018,0.767792,0.000126
3,LogReg,IMU_ONLY,0.357405,0.008640,0.639138,0.008640
4,kNN,IMU_ONLY,0.335120,0.010677,0.772294,0.003770
5,CatBoost,IMU_ONLY,0.319251,0.005429,0.772677,0.002239
6,LogReg,TOTAL,0.386057,0.003570,0.633662,0.001441
7,kNN,TOTAL,0.346712,0.008934,0.772443,0.002801
8,CatBoost,TOTAL,0.312686,0.001845,0.770581,0.000857


### Train-test split

In [41]:
# ================================================================
# 0)  train ↔ test split by participant
#     ---------------------------------
# subjects_total : (N,)  int   1 … 25 for every window/row
# ================================================================
train_mask = subjects_total <= 20
test_mask  = subjects_total > 20

X_eeg_tr, X_eeg_te = eeg_feat[train_mask], eeg_feat[test_mask]
X_imu_tr, X_imu_te = imu_feat[train_mask], imu_feat[test_mask]
X_feat_tr, X_feat_te = X_feat[train_mask], X_feat[test_mask]
y_tr, y_te = y_total[train_mask], y_total[test_mask]

print(f"train={len(y_tr)}  test={len(y_te)} windows")

train=14935  test=3891 windows


In [55]:
%%time
MODEL_NAMES = ("LogReg", "kNN", "CatBoost")


def instantiate_model(model_nm):
    assert model_nm in MODEL_NAMES

    if model_nm == "LogReg":
        return make_pipeline(
            StandardScaler(),
            LogisticRegression(
                max_iter=2000,
                class_weight='balanced'
            )
        )
    elif model_nm == "kNN":
        return make_pipeline(
            StandardScaler(),
            KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        )
    elif model_nm == "CatBoost":
        return CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    else:
        return None


datasets = {
    'EEG_ONLY': (X_eeg_tr, X_eeg_te),
    'IMU_ONLY': (X_imu_tr, X_imu_te),
    'TOTAL': (X_feat_tr, X_feat_te)
}
baseline_results_tr_te = []

for dataset_nm, (temp_tr, temp_te) in tqdm(datasets.items()):
    for model_nm in tqdm(MODEL_NAMES):
        model = instantiate_model(model_nm).fit(temp_tr, y_tr)
        model.fit(temp_tr, y_tr)
        y_pred = model.predict(temp_te)
        
        row = {
            'model': model_nm,
            'data': dataset_nm,
            'f1_macro': f1_score(y_te, y_pred, average='macro'),
            'f1_weighted': f1_score(y_te, y_pred, average='weighted'),
        }
        baseline_results_tr_te.append(row)
    gc.collect()
    
baseline_results_tr_te = pd.DataFrame(baseline_results_tr_te)
baseline_results_tr_te.to_csv('../experiment-results/human-player/baseline_tr_te.csv', index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 38min 32s, sys: 2min 26s, total: 40min 59s
Wall time: 2min 21s


In [56]:
baseline_results_tr_te

,model,data,f1_macro,f1_weighted
0,LogReg,EEG_ONLY,0.351678,0.666268
1,kNN,EEG_ONLY,0.324475,0.810013
2,CatBoost,EEG_ONLY,0.311762,0.821590
3,LogReg,IMU_ONLY,0.326279,0.646842
4,kNN,IMU_ONLY,0.325246,0.814854
5,CatBoost,IMU_ONLY,0.314076,0.821655
6,LogReg,TOTAL,0.346474,0.655367
7,kNN,TOTAL,0.330003,0.814501
8,CatBoost,TOTAL,0.314275,0.822065


## Causal dim.reduction methods

### PurePCA

In [82]:
class PurePCA:
    """
    Two *independent* PCA projections (no causal block, no lag scan).

    Parameters
    ----------
    d_x : int   – # components kept for X
    d_y : int   – # components kept for Y
    """
    def __init__(self, d_x=10, d_y=10):
        self.d_x = d_x
        self.d_y = d_y
        self.pca_x_ = None
        self.pca_y_ = None

    # ---------- scikit-style API ----------
    def fit(self, X, Y=None):
        """Fit two separate PCAs."""
        self.pca_x_ = PCA(n_components=self.d_x, random_state=SEED).fit(X)
        self.pca_y_ = PCA(n_components=self.d_y, random_state=SEED).fit(Y)
        return self

    def transform(self, X, Y):
        Zx = self.pca_x_.transform(X)
        Zy = self.pca_y_.transform(Y)
        return (Zx, Zy)

    def fit_transform(self, X, Y):
        self.fit(X, Y)
        return self.transform(X, Y)

    # optional reconstructions
    def reconstruct_x(self, X):
        return self.pca_x_.inverse_transform(self.pca_x_.transform(X))

    def reconstruct_y(self, Y):
        return self.pca_y_.inverse_transform(self.pca_y_.transform(Y))


### PureCCA

In [83]:
class PureCCA:
    """
    Keep **only** the canonical sub-space (no reconstructive block).

    Parameters
    ----------
    d_c : int  – # canonical dimensions to keep
    scale : bool – passed straight to sklearn.CCA
    """
    def __init__(self, d_x=5, d_y=5):
        self.d_x = d_x
        self.d_y = d_y
        self.cca_ = None

    # ---------- scikit-style API ----------
    def fit(self, X, Y):
        self.cca_ = CCA(
            n_components=min(self.d_x, self.d_y),
            scale=True,
            max_iter=1000,
            tol=3e-5)\
        .fit(X, Y)
        return self

    def transform(self, X, Y):
        Zx, Zy = self.cca_.transform(X, Y)
        return (Zx, Zy)

    def fit_transform(self, X, Y):
        self.fit(X, Y)
        return self.transform(X, Y, split)

    def reconstruct_x(self, X):
        Zx = self.cca_.transform(X)
        X_recon = Zx @ self.cca_.x_weights_.T
        return X_recon

    def reconstruct_y(self, Y):
        _, Wy = self.cca_.transform(
            np.zeros(len(Y), len(self.cca_.x_loadings_)),
            Y
        )
        Y_recon = Wy @ self.cca_.y_weights_.T
        return Y_recon

### CaSCA

In [13]:
class CaSCA:
    """
    CaSCA pipeline as a scikit-learn style transformer.

    Parameters
    ----------
    lag_set : sequence of int
        Candidate lags to scan for causal delay.
    d_c : int
        Number of causal dimensions.
    d_hid : int
        Total hidden dimensionality (causal + reconstructive).
    """
    def __init__(self, lag_set, d_c=1, d_hid=3):
        if isinstance(d_hid, tuple):
            assert d_c <= min(d_hid), "Causal dim should be <= hidden dim."
        else:
            assert d_c <= d_hid, "Causal dim should be <= hidden dim."
        self.lag_set = lag_set
        self.d_c = d_c
        self.d_hid = d_hid
        self.cca_ = None
        self.pca_x_ = None
        self.pca_y_ = None
        self.tau_star_ = None

    def _get_lags(self, X, Y):
        T = len(X)
        X_tau = X[:T - self.tau_star_] if self.tau_star_ else X
        Y_tau = Y[self.tau_star_:] if self.tau_star_ else Y
        return X_tau, Y_tau
        
    def _scan_lagged_cca(
        self, X: np.ndarray, Y: np.ndarray
    ):
        """Return τ⋆ and its first canonical correlation."""
        if len(self.lag_set) == 1:
            self.tau_star_ = self.lag_set[0]
            return
            
        best_tau, best_rho = None, -np.inf
        T = len(X)
        for tau in self.lag_set:
            Xτ = X[:T - tau] if tau else X
            Yτ = Y[tau:] if tau else Y
            cca = CCA(n_components=1, scale=True)
            u, v = cca.fit_transform(Xτ, Yτ)
            rho = np.corrcoef(u[:, 0], v[:, 0])[0, 1]
            if rho > best_rho:
                best_tau, best_rho = tau, rho

        self.tau_star_ = best_tau

    def _deflate(self, Xc, Yc, cca_proj_x, cca_proj_y, return_padded=False):
        Zc = np.pad(cca_proj_x, ((0, self.tau_star_), (0, 0)), mode="edge")
        Wc = np.pad(cca_proj_y, ((self.tau_star_, 0), (0, 0)), mode="edge")
        
        X_res = Xc - Zc @ self.cca_.x_weights_.T
        Y_res = Yc - Wc @ self.cca_.y_weights_.T

        if return_padded:
            return X_res, Y_res, Zc, Wc
        else:
            return X_res, Y_res
    
    def fit(self, X, Y):
        # center
        T = len(X)
        Xc = X - X.mean(axis=0)
        Yc = Y - Y.mean(axis=0)
        
        # scan lag
        self._scan_lagged_cca(X, Y)
                
        # fit CCA causal block
        X_tau, Y_tau = self._get_lags(Xc, Yc)
        self.cca_ = CCA(n_components=self.d_c, scale=True, max_iter=1000, tol=3e-05)
        U, V = self.cca_.fit_transform(X_tau, Y_tau)

        # deflate
        X_res, Y_res = self._deflate(Xc, Yc, U, V)
        
        # reconstruction PCA
        if isinstance(self.d_hid, (int, np.int32, np.int64)):
            d_res = self.d_hid - self.d_c
            self.pca_x_ = PCA(n_components=d_res, random_state=SEED)
            self.pca_y_ = PCA(n_components=d_res, random_state=SEED)
        elif isinstance(self.d_hid, tuple):
            d_res_x = self.d_hid[0] - self.d_c
            d_res_y = self.d_hid[1] - self.d_c
            self.pca_x_ = PCA(n_components=d_res_x, random_state=SEED)
            self.pca_y_ = PCA(n_components=d_res_y, random_state=SEED)
            
        self.pca_x_.fit(X_res)
        self.pca_y_.fit(Y_res)
        
        return self

    def transform(self, X, Y, split=False):
        """
        Transform new data into [causal, reconstructive] latents.

        Parameters
        ----------
        X, Y : array-like, shape (T, n_x), (T, n_y)
        split : bool
            If True, return Z_c, Z_r, W_c, W_r; else concatenate Z and reconstructive.
        """
        Xc = X - X.mean(axis=0)
        Yc = Y - Y.mean(axis=0)
        T, _ = Xc.shape
        
        # lagged
        X_tau, Y_tau = self._get_lags(Xc, Yc)
        U, V = self.cca_.transform(X_tau, Y_tau)
        
        # deflate
        X_res, Y_res, Zc, Wc = self._deflate(Xc, Yc, U, V, return_padded=True)
        
        Zr = self.pca_x_.transform(X_res)
        Wr = self.pca_y_.transform(Y_res)
        
        if split:
            return Zc, Zr, Wc, Wr
        else:
            return np.hstack([Zc, Zr]), np.hstack([Wc, Wr])

    def fit_transform(self, X, Y, split=False):
        self.fit(X, Y)
        return self.transform(X, Y, split)

    def reconstruct_x(self, X: np.ndarray) -> np.ndarray:
        """
        Given a fitted CaSCA model and original X, reconstruct X from latents:
           X_hat = mean(X) + Z_c @ A_c^T + Z_r @ PCA.components_
        """
        Xc = X - X.mean(axis=0)
        T, _ = Xc.shape
        tau = self.tau_star_
        X_tau = Xc[:T - tau] if tau else Xc
        
        U_c = self.cca_.transform(X_tau)
        Zc = np.pad(U_c, ((0, tau), (0, 0)), mode="edge")
        X_res = Xc - Zc @ self.cca_.x_weights_.T
        Zr = self.pca_x_.transform(X_res)
        Xc_hat = Zc @ self.cca_.x_weights_.T + Zr @ self.pca_x_.components_
        X_hat = Xc_hat + X.mean(axis=0)
        return X_hat

    def reconstruct_y(self, Y: np.ndarray) -> np.ndarray:
        """
        Reconstruct Y analogous to X reconstruction.
        """
        Yc = Y - Y.mean(axis=0)
        T, _ = Yc.shape
        tau = self.tau_star_
        Y_tau = Yc[:T - tau] if tau else Xc
        
        _, V_c = self.cca_.transform(
            np.zeros((len(Y_tau), len(self.cca_.x_loadings_))),
            Y_tau
        )
        Wc = np.pad(V_c, ((tau, 0), (0, 0)), mode="edge")
        Y_res = Yc - Wc @ self.cca_.y_weights_.T
        Wr = self.pca_y_.transform(Y_res)
        Yc_hat = Wc @ self.cca_.y_weights_.T + Wr @ self.pca_y_.components_
        Y_hat = Yc_hat + Y.mean(axis=0)
        return Y_hat

### ICA-LinGAM

In [ ]:
class CrossICA_LiNGAM:
    """
    1. ICA on [X|Y]          (stat. independence)
    2. DirectLiNGAM on ICs   (keep sources that drive across blocks)
    Parameters
    ----------
    random_state : int | None
    """
    def __init__(self, random_state=0):
        self.random_state = random_state
        self.ica_ = None
        self.lingam_ = None
        self.keep_idx_ = None          # indices of causal sources
        self.W_ = None                 # mixing sub-matrix ( (p+q) × d )

    # ------------------------------------------------------------
    def fit(self, X, Y):
        XY = np.hstack([X, Y])
        self.ica_ = FastICA(n_components="auto",
                            whiten="unit-variance",
                            random_state=self.random_state)
        S = self.ica_.fit_transform(XY)               # (N, k)

        # run LiNGAM on sources
        self.lingam_ = lingam.DirectLiNGAM(random_state=self.random_state)
        self.lingam_.fit(S)
        B = self.lingam_.adjacency_matrix_            # (k,k)

        # map each source to "lives mostly in X or Y"
        load = self.ica_.mixing_                      # (p+q, k)
        p = X.shape[1]
        x_mask = np.arange(p)
        y_mask = np.arange(p, p + Y.shape[1])

        src_on_X = np.where(np.abs(load[x_mask]).sum(0) >
                            np.abs(load[y_mask]).sum(0))[0]
        src_on_Y = np.setdiff1d(np.arange(load.shape[1]), src_on_X)

        # keep sources that drive across blocks
        keep_in_X = _select_by_out_degree(
            B, src_on_X, src_on_Y)                    # X→Y
        keep_in_Y = _select_by_out_degree(
            B, src_on_Y, src_on_X)                    # Y→X
        self.keep_idx_ = np.sort(np.concatenate([keep_in_X, keep_in_Y]))
        self.W_ = load[:, self.keep_idx_]             # projection

        return self

    # ------------------------------------------------------------
    def transform(self, X, Y, split=True):
        XY = np.hstack([X, Y])
        S  = self.ica_.transform(XY)[:, self.keep_idx_]  # (N, d)
        # split coords back to X/Y by loadings magnitude
        p = X.shape[1]
        x_load = np.abs(self.W_[:p]).sum(0)
        y_load = np.abs(self.W_[p:]).sum(0)
        Zx = S[:, x_load >= y_load]
        Zy = S[:, y_load >  x_load]
        return (Zx, Zy) if split else (S,)

In [14]:
# ica_lg = lingam.ICALiNGAM(random_state, max_iter)
# ica_lg.fit(X)

### Granger-PCA 

In [15]:
class BiGrangerPCA:
    """
    PCA in X and Y  ➜  rank PCs by Granger F-stat X→Y (or Y→X)
    Parameters
    ----------
    n_x, n_y : #PCs to keep initially
    lag_order : VAR lag
    top_k : final #coords to keep from each side
    direction : {"X2Y", "Y2X"}  – causal direction of interest
    """
    def __init__(self, n_x=10, n_y=10, lag_order=5,
                 top_k=5, direction="X2Y", random_state=0):
        self.n_x, self.n_y = n_x, n_y
        self.lag_order = lag_order
        self.top_k = top_k
        self.direction = direction.upper()
        self.random_state = random_state
        self.pca_x_ = None
        self.pca_y_ = None
        self.order_ = None                 # indices of kept PCs

    # ------------------------------------------------------------
    def fit(self, X, Y):
        self.pca_x_ = PCA(self.n_x, random_state=self.random_state).fit(X)
        self.pca_y_ = PCA(self.n_y, random_state=self.random_state).fit(Y)
        Zx = self.pca_x_.transform(X)
        Zy = self.pca_y_.transform(Y)

        Z  = np.hstack([Zx, Zy])
        var = sm.tsa.VAR(Z).fit(self.lag_order)

        f_stats = []
        if self.direction == "X2Y":
            for k in range(self.n_x):
                f = var.test_causality([k],
                        list(range(self.n_x, self.n_x + self.n_y)),
                        kind='f').test_statistic
                f_stats.append(f)
            self.order_ = np.argsort(f_stats)[::-1][:self.top_k]
        else:  # Y2X
            for k in range(self.n_y):
                f = var.test_causality(
                        [self.n_x + k], list(range(self.n_x)),
                        kind='f').test_statistic
                f_stats.append(f)
            self.order_ = np.argsort(f_stats)[::-1][:self.top_k]

        return self

    # ------------------------------------------------------------
    def transform(self, X, Y, split=True):
        Zx = self.pca_x_.transform(X)
        Zy = self.pca_y_.transform(Y)

        if self.direction == "X2Y":
            Zx_c = Zx[:, self.order_]
            return (Zx_c, Zy) if split else (np.hstack([Zx_c, Zy]),)
        else:
            Zy_c = Zy[:, self.order_]
            return (Zx, Zy_c) if split else (np.hstack([Zx, Zy_c]),)

### PCMCI + Eigen Projection

In [16]:
class PCMCIeigen:
    """
    1. PCMCI on [X|Y]
    2. build bipartite weight matrix of lagged cross-causal edges
    3. SVD / Laplacian eigenvectors → matched coordinates
    Parameters
    ----------
    tau_max : int   (maximum lag for PCMCI)
    k_keep  : int   (#eigenvectors per side to keep)
    """
    def __init__(self, tau_max=8, k_keep=4, alpha=0.01):
        self.tau_max = tau_max
        self.k_keep = k_keep
        self.alpha = alpha
        self.Ux_, self.Vy_ = None, None   # projection bases

    # ------------------------------------------------------------
    def fit(self, X, Y):
        XY = np.hstack([X, Y])
        p, q = X.shape[1], Y.shape[1]

        df  = dp.DataFrame(XY)
        par = independence_tests.ParCorr()
        pc  = pcmci.PCMCI(df, par_corr=par, verbosity=0)
        res = pc.run_pcmci(tau_max=self.tau_max, alpha_level=self.alpha)

        A = pc._return_graph(res, alpha_level=self.alpha)  # (p+q,p+q, τ+1)
        Wxy = np.abs(A[:p, p:, 1:]).sum(axis=2)            # cross X→Y
        # SVD of bipartite weight
        U, S, Vt = np.linalg.svd(Wxy, full_matrices=False)
        self.Ux_ = U[:, :self.k_keep]                      # (p,k)
        self.Vy_ = Vt[:self.k_keep].T                      # (q,k)
        return self

    # ------------------------------------------------------------
    def transform(self, X, Y, split=True):
        Zx = X @ self.Ux_
        Zy = Y @ self.Vy_
        return (Zx, Zy) if split else (np.hstack([Zx, Zy]),)


### Quality criteria

In [19]:
def compute_vif(X: np.ndarray) -> pd.DataFrame:
    """
    Compute Variance Inflation Factor (VIF) for each feature in X.
    
    Parameters
    ----------
    X : ndarray of shape (n_samples, n_features)
        Input data matrix (e.g., latent space representation).
    
    Returns
    -------
    vif_df : pandas DataFrame
        DataFrame with columns ['feature', 'VIF'].
    """
    n_samples, n_features = X.shape
    vif_values = []
    for j in range(n_features):
        # Define feature j as target, others as predictors
        X_j = X[:, j]
        X_others = np.delete(X, j, axis=1)
        
        # Fit linear regression of X_j on X_others
        model = LinearRegression().fit(X_others, X_j)
        r2_j = model.score(X_others, X_j)
        vif_j = 1.0 / (1.0 - r2_j) if r2_j < 1.0 else np.inf
        vif_values.append(vif_j)

    return max(vif_values)

def compute_cond_number(X: np.ndarray) -> float:
    """
    Compute the condition number of X^T X.
    
    Parameters
    ----------
    X : ndarray of shape (n_samples, n_features)
        Input data matrix.
    
    Returns
    -------
    cond_number : float
        Condition number (ratio of largest to smallest singular value).
    """
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    cond_number = np.log(s.max() / s.min())
    return cond_number


def compute_rmse(X_true: np.ndarray, X_pred: np.ndarray) -> float:
    """
    Compute Root Mean Squared Error between X_true and X_pred.
    """
    return np.sqrt(np.mean((X_true - X_pred) ** 2))


def compute_explained_variance(X_true: np.ndarray, X_pred: np.ndarray) -> float:
    """
    Compute the fraction of variance in X_true explained by X_pred.
    """
    var_true = np.var(X_true, axis=0).sum()
    var_resid = np.var(X_true - X_pred, axis=0).sum()
    return 1 - (var_resid / var_true)

## Prediction pipeline (for PurePCA and PureCCA)

In [84]:
lag_set = [0]               # ignore lag scan in this experiment
# grid_d_c  = [1, 3, 5, 7, 9, 11, 13, 15, 20, 30, 40]
grid_d_hid = [2, 4, 6, 8, 10, 12, 14, 16, 30, 50, 70, 90]


def make_clfs():
    return {
        "LogReg": LogisticRegression(
            max_iter=2000,
            class_weight='balanced'
        ),
        "kNN": KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        "CatBoost": CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            devices='0',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    }

# helper: run one grid point
def run_grid_point(d_hid, dim_red_model_cls):
    # ❶  fit CaSCA on *train* only
    dr_model = dim_red_model_cls(d_hid)
    dr_model.fit(X_eeg_tr, X_imu_tr)

    Ztr, Wtr = dr_model.transform(X_eeg_tr, X_imu_tr)
    Zte, Wte = dr_model.transform(X_eeg_te, X_imu_te)

    FEAT_tr = np.hstack([Ztr, Wtr])
    FEAT_te = np.hstack([Zte, Wte])

    # ❷  embedding diagnostics (train set)
    vif   = compute_vif(FEAT_tr)
    cnum  = compute_cond_number(FEAT_tr)
    Xrec  = dr_model.reconstruct_x(X_eeg_tr)
    rmse  = compute_rmse(X_eeg_tr, Xrec)
    ev    = compute_explained_variance(X_eeg_tr, Xrec)

    row = dict(d_hid=d_hid,
               VIF=vif, log_cond_XTX=cnum,
               RMSE_recon=rmse, ExplVar=ev)

    # ❸  classifiers
    for name, clf in make_clfs().items():
        clf.fit(FEAT_tr, y_tr)
        y_pred = clf.predict(FEAT_te)

        row[f"{name}_F1w"] = f1_score(y_te, y_pred, average="weighted")
        row[f"{name}_F1m"] = f1_score(y_te, y_pred, average="macro")

    return row

# ================================================================
# 3)  Run over grid & collect
# ================================================================
results = []
for d_hid in tqdm(grid_d_hid):
    for dim_red_model_cls in (PurePCA, PureCCA):
        row = run_grid_point(d_hid, dim_red_model_cls)
        row['din_red_model'] = type(dim_red_model_cls(1)).__name__
        results.append(row)
    gc.collect()

df = pd.DataFrame(results)
print("\n===== summary =====")
print(df.to_string(index=False))

  0%|          | 0/12 [00:00<?, ?it/s]


===== summary =====
 d_hid      VIF  log_cond_XTX  RMSE_recon   ExplVar  LogReg_F1w  LogReg_F1m  kNN_F1w  kNN_F1m  CatBoost_F1w  CatBoost_F1m din_red_model
     2 1.217931      2.445204    0.145508  0.347674    0.716432    0.351534 0.814274 0.338152      0.822435      0.314425       PurePCA
     2 3.414813      8.282478    0.180449 -0.000043    0.717255    0.344524 0.811348 0.324482      0.821415      0.313985       PureCCA
     4 1.219511      2.558359    0.140923  0.388138    0.721258    0.356644 0.811931 0.331186      0.821805      0.314159       PurePCA
     4 3.415327      8.826862    0.180450 -0.000049    0.696617    0.340955 0.812972 0.321667      0.822597      0.320912       PureCCA
     6 1.225162      2.635144    0.137213  0.419928    0.710594    0.352038 0.813290 0.338758      0.823645      0.323700       PurePCA
     6 3.415327      8.826862    0.180449 -0.000041    0.690671    0.345252 0.814312 0.331160      0.822268      0.323466       PureCCA
     8 1.226387      2.7158

In [86]:
df.to_csv('../experiment-results/human-player/baseline_other.csv', index=False)

## Original space

In [155]:
del eeg_feat, imu_feat, X_feat

In [158]:
%%time
def add_acceleration(arr):
    # arr: (N, C, T) → output shape (N, C+1, T)
    acceleration = np.sqrt(np.sum(np.square(arr[:, -3:, :]), axis=1, keepdims=True))
    arr_stacked = np.concatenate((arr, acceleration), axis=1)
    return arr_stacked

sfreq = 250
win_len = 20
step = 20
ends = sfreq - np.arange(0, eeg_total.shape[2] - win_len + 1, step)
n_win = len(ends)

# (N, C, n_win, win_len)  ->  (N, C, n_win) -> (N, C*n_win)
eeg_feat = np.stack(
    [eeg_total[:, :, e-win_len:e] for e in ends],
    axis=2
).mean(axis=-1).reshape(len(eeg_total), -1)

imu_total_with_acc = add_acceleration(imu_total)
imu_feat = np.stack(
    [imu_total_with_acc[:, :, e-win_len:e] for e in ends],
    axis=2
).mean(axis=-1).reshape(len(imu_total), -1)

X_feat = np.hstack([eeg_feat, imu_feat])
del imu_total_with_acc

CPU times: user 2.47 s, sys: 1.65 s, total: 4.12 s
Wall time: 4.48 s


In [159]:
eeg_feat.shape, imu_feat.shape, X_feat.shape

((18826, 1428), (18826, 144), (18826, 1572))

**Train-test split**

In [160]:
train_mask = subjects_total <= 20
test_mask  = subjects_total > 20

X_eeg_tr, X_eeg_te = eeg_feat[train_mask], eeg_feat[test_mask]
X_imu_tr, X_imu_te = imu_feat[train_mask], imu_feat[test_mask]
X_feat_tr, X_feat_te = X_feat[train_mask], X_feat[test_mask]
y_tr, y_te = y_total[train_mask], y_total[test_mask]

print(f"train={len(y_tr)}  test={len(y_te)} windows")

train=14935  test=3891 windows


### EEG vs IMU vs both

In [163]:
%%time
MODEL_NAMES = ("LogReg", "kNN", "CatBoost")


def instantiate_model(model_nm):
    assert model_nm in MODEL_NAMES

    if model_nm == "LogReg":
        return make_pipeline(
            StandardScaler(),
            LogisticRegression(
                max_iter=2000,
                class_weight='balanced'
            )
        )
    elif model_nm == "kNN":
        return make_pipeline(
            StandardScaler(),
            KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        )
    elif model_nm == "CatBoost":
        return CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    else:
        return None


datasets = {
    'EEG_ONLY': (X_eeg_tr, X_eeg_te),
    'IMU_ONLY': (X_imu_tr, X_imu_te),
    'TOTAL': (X_feat_tr, X_feat_te)
}
original_results_tr_te = []

for dataset_nm, (temp_tr, temp_te) in tqdm(datasets.items()):
    for model_nm in tqdm(MODEL_NAMES):
        model = instantiate_model(model_nm).fit(temp_tr, y_tr)
        model.fit(temp_tr, y_tr)
        y_pred = model.predict(temp_te)
        
        row = {
            'model': model_nm,
            'data': dataset_nm,
            'f1_macro': f1_score(y_te, y_pred, average='macro'),
            'f1_weighted': f1_score(y_te, y_pred, average='weighted'),
        }
        original_results_tr_te.append(row)
    gc.collect()
    
original_results_tr_te = pd.DataFrame(original_results_tr_te)
original_results_tr_te.to_csv('../experiment-results/human-player/original_tr_te.csv', index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 44min 28s, sys: 3min 31s, total: 47min 59s
Wall time: 2min 54s


### CaSCA grid

In [165]:
lag_set   = [0]               # ignore lag scan in this experiment
grid_d_c  = [1, 3, 5, 7, 9, 11, 13, 15, 20, 30, 40]
grid_dhid = [2, 4, 6, 8, 10, 12, 14, 16, 30, 50, 70, 90]

# ================================================================
# 1)  classifier factory (unchanged except seeds)
# ================================================================
def make_clfs():
    return {
        "LogReg": LogisticRegression(
            max_iter=2000,
            class_weight='balanced'
        ),
        "kNN": KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        "CatBoost": CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    }

# ------------------------------------------------- helper: run one grid point
def run_grid_point(d_c, d_hid):
    print(f"  ➜ d_c={d_c:2d}  d_hid={d_hid:2d}")

    # ❶  fit CaSCA on *train* only
    casca = CaSCA(lag_set=[0], d_c=d_c, d_hid=(d_hid, min(d_hid, X_imu_tr.shape[1])))
    casca.fit(X_eeg_tr, X_imu_tr)

    Ztr, Wtr = casca.transform(X_eeg_tr, X_imu_tr)
    Zte, Wte = casca.transform(X_eeg_te, X_imu_te)

    FEAT_tr = np.hstack([Ztr, Wtr])
    FEAT_te = np.hstack([Zte, Wte])

    # ❷  embedding diagnostics (train set)
    vif   = compute_vif(FEAT_tr)
    cnum  = compute_cond_number(FEAT_tr)
    Xrec  = casca.reconstruct_x(X_eeg_tr)
    rmse  = compute_rmse(X_eeg_tr, Xrec)
    ev    = compute_explained_variance(X_eeg_tr, Xrec)

    row = dict(d_c=d_c, d_hid=d_hid,
               VIF=vif, log_cond_XTX=cnum,
               RMSE_recon=rmse, ExplVar=ev)

    # ❸  classifiers
    for name, clf in make_clfs().items():
        clf.fit(FEAT_tr, y_tr)
        y_pred = clf.predict(FEAT_te)

        row[f"{name}_F1w"] = f1_score(y_te, y_pred, average="weighted")
        row[f"{name}_F1m"] = f1_score(y_te, y_pred, average="macro")

    return row

# ================================================================
# 3)  Run over grid & collect
# ================================================================
casca_orig_results = []
for d_c in tqdm(grid_d_c):
    for d_hid in tqdm(grid_dhid):
        if d_c > d_hid:                 # skip invalid combos
            continue
        casca_orig_results.append(run_grid_point(d_c, d_hid))
    gc.collect()

casca_orig_df = pd.DataFrame(casca_orig_results)
casca_orig_df.to_csv('../experiment-results/human-player/original_casca.csv', index=False)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  ➜ d_c= 1  d_hid= 2
  ➜ d_c= 1  d_hid= 4
  ➜ d_c= 1  d_hid= 6
  ➜ d_c= 1  d_hid= 8
  ➜ d_c= 1  d_hid=10
  ➜ d_c= 1  d_hid=12
  ➜ d_c= 1  d_hid=14
  ➜ d_c= 1  d_hid=16
  ➜ d_c= 1  d_hid=30
  ➜ d_c= 1  d_hid=50
  ➜ d_c= 1  d_hid=70
  ➜ d_c= 1  d_hid=90


  0%|          | 0/12 [00:00<?, ?it/s]

  ➜ d_c= 3  d_hid= 4
  ➜ d_c= 3  d_hid= 6
  ➜ d_c= 3  d_hid= 8
  ➜ d_c= 3  d_hid=10
  ➜ d_c= 3  d_hid=12
  ➜ d_c= 3  d_hid=14
  ➜ d_c= 3  d_hid=16
  ➜ d_c= 3  d_hid=30
  ➜ d_c= 3  d_hid=50
  ➜ d_c= 3  d_hid=70
  ➜ d_c= 3  d_hid=90


...

  ➜ d_c=20  d_hid=30
  ➜ d_c=20  d_hid=50
  ➜ d_c=20  d_hid=70
  ➜ d_c=20  d_hid=90


  0%|          | 0/12 [00:00<?, ?it/s]

  ➜ d_c=30  d_hid=30
  ➜ d_c=30  d_hid=50
  ➜ d_c=30  d_hid=70
  ➜ d_c=30  d_hid=90


  0%|          | 0/12 [00:00<?, ?it/s]

  ➜ d_c=40  d_hid=50
  ➜ d_c=40  d_hid=70
  ➜ d_c=40  d_hid=90


### PurePCA and PureCCA

In [164]:
lag_set = [0]               # ignore lag scan in this experiment
grid_d_hid = [2, 4, 6, 8, 10, 12, 14, 16, 30, 50, 70, 90]


def make_clfs():
    return {
        "LogReg": LogisticRegression(
            max_iter=2000,
            class_weight='balanced'
        ),
        "kNN": KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        "CatBoost": CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            devices='0',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    }

# helper: run one grid point
def run_grid_point(d_hid, dim_red_model_cls):
    # ❶  fit CaSCA on *train* only
    dr_model = dim_red_model_cls(d_hid)
    dr_model.fit(X_eeg_tr, X_imu_tr)

    Ztr, Wtr = dr_model.transform(X_eeg_tr, X_imu_tr)
    Zte, Wte = dr_model.transform(X_eeg_te, X_imu_te)

    FEAT_tr = np.hstack([Ztr, Wtr])
    FEAT_te = np.hstack([Zte, Wte])

    # ❷  embedding diagnostics (train set)
    vif   = compute_vif(FEAT_tr)
    cnum  = compute_cond_number(FEAT_tr)
    Xrec  = dr_model.reconstruct_x(X_eeg_tr)
    rmse  = compute_rmse(X_eeg_tr, Xrec)
    ev    = compute_explained_variance(X_eeg_tr, Xrec)

    row = dict(d_hid=d_hid,
               VIF=vif, log_cond_XTX=cnum,
               RMSE_recon=rmse, ExplVar=ev)

    # ❸  classifiers
    for name, clf in make_clfs().items():
        clf.fit(FEAT_tr, y_tr)
        y_pred = clf.predict(FEAT_te)

        row[f"{name}_F1w"] = f1_score(y_te, y_pred, average="weighted")
        row[f"{name}_F1m"] = f1_score(y_te, y_pred, average="macro")

    return row

# ================================================================
# 3)  Run over grid & collect
# ================================================================
orig_pure_results = []
for d_hid in tqdm(grid_d_hid):
    for dim_red_model_cls in (PurePCA, PureCCA):
        row = run_grid_point(d_hid, dim_red_model_cls)
        row['din_red_model'] = type(dim_red_model_cls(1)).__name__
        orig_pure_results.append(row)
    gc.collect()

orig_pure_df = pd.DataFrame(orig_pure_results)
orig_pure_df.to_csv('../experiment-results/human-player/original_other.csv', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]

## Trajectory space

In [89]:
del eeg_feat, imu_feat, X_feat

In [181]:
%%time
sfreq = 250
win_len = int(0.3 * sfreq)
step = int(0.2 * sfreq)
ends = sfreq - np.arange(0, eeg_total.shape[2] - win_len + 1, step)
n_win = len(ends)

# (N, n_win, C, win_len)  ->  (N*n_win, C, win_len)
eeg_subwin = np.stack(
    [eeg_total[:, :, e-win_len:e] for e in ends],
    axis=1
).reshape(-1, eeg_total.shape[1], win_len)
y_subwin = np.repeat(y_total, n_win)    # same label for every sub-window

xd = XdawnCovariances(nfilter=5, estimator='oas')
ts = TangentSpace(metric='riemann')
cov = xd.fit_transform(eeg_subwin, y_subwin)           # (N*n_win, P, P)
eeg_ts = ts.fit_transform(cov)                         # (N*n_win, D_ts)
eeg_feat = eeg_ts.reshape(-1, n_win * eeg_ts.shape[1]) # (N, 4·D_ts)

imu_subwin = np.stack(
    [imu_total[:, :, e-win_len:e] for e in ends],
    axis=1
).reshape(-1, imu_total.shape[1], win_len)
imu_values = imu_stats(imu_subwin)
imu_feat = imu_values.reshape(-1, n_win * imu_values.shape[1]) # (N, (C+1)*4)

X_feat = np.hstack([eeg_feat, imu_feat])         # (N, 4·D_ts + (C+1)*4)

del eeg_subwin, cov, eeg_ts, imu_subwin, imu_values

CPU times: user 2min 40s, sys: 2min 53s, total: 5min 34s
Wall time: 2min 25s


In [182]:
eeg_feat.shape, imu_feat.shape, X_feat.shape

((18826, 1860), (18826, 192), (18826, 2052))

**Train-test split**

In [184]:
train_mask = subjects_total <= 20
test_mask  = subjects_total > 20

X_eeg_tr, X_eeg_te = eeg_feat[train_mask], eeg_feat[test_mask]
X_imu_tr, X_imu_te = imu_feat[train_mask], imu_feat[test_mask]
X_feat_tr, X_feat_te = X_feat[train_mask], X_feat[test_mask]
y_tr, y_te = y_total[train_mask], y_total[test_mask]

print(f"train={len(y_tr)}  test={len(y_te)} windows")

train=14935  test=3891 windows


### EEG vs IMU vs both

In [185]:
%%time
MODEL_NAMES = ("LogReg", "kNN", "CatBoost")


def instantiate_model(model_nm):
    assert model_nm in MODEL_NAMES

    if model_nm == "LogReg":
        return make_pipeline(
            StandardScaler(),
            LogisticRegression(
                max_iter=2000,
                class_weight='balanced'
            )
        )
    elif model_nm == "kNN":
        return make_pipeline(
            StandardScaler(),
            KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        )
    elif model_nm == "CatBoost":
        return CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    else:
        return None


datasets = {
    'EEG_ONLY': (X_eeg_tr, X_eeg_te),
    'IMU_ONLY': (X_imu_tr, X_imu_te),
    'TOTAL': (X_feat_tr, X_feat_te)
}
traj_results_tr_te = []

for dataset_nm, (temp_tr, temp_te) in tqdm(datasets.items()):
    for model_nm in tqdm(MODEL_NAMES):
        model = instantiate_model(model_nm).fit(temp_tr, y_tr)
        model.fit(temp_tr, y_tr)
        y_pred = model.predict(temp_te)
        
        row = {
            'model': model_nm,
            'data': dataset_nm,
            'f1_macro': f1_score(y_te, y_pred, average='macro'),
            'f1_weighted': f1_score(y_te, y_pred, average='weighted'),
        }
        traj_results_tr_te.append(row)
    gc.collect()
    
traj_results_tr_te = pd.DataFrame(traj_results_tr_te)
traj_results_tr_te.to_csv('../experiment-results/human-player/trajectory_tr_te.csv', index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 1h 13min 46s, sys: 8min 29s, total: 1h 22min 15s
Wall time: 4min 55s


### CasCA grid

In [187]:
lag_set   = [0]               # ignore lag scan in this experiment
grid_d_c  = [1, 3, 5, 7, 9, 11, 13, 15, 20, 30, 40]
grid_dhid = [2, 4, 6, 8, 10, 12, 14, 16, 30, 50, 70, 90]

# ================================================================
# 1)  classifier factory (unchanged except seeds)
# ================================================================
def make_clfs():
    return {
        "LogReg": LogisticRegression(
            max_iter=2000,
            class_weight='balanced'
        ),
        "kNN": KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        "CatBoost": CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    }

# ------------------------------------------------- helper: run one grid point
def run_grid_point(d_c, d_hid):
    print(f"  ➜ d_c={d_c:2d}  d_hid={d_hid:2d}")

    # ❶  fit CaSCA on *train* only
    casca = CaSCA(lag_set=[0], d_c=d_c, d_hid=(d_hid, min(d_hid, X_imu_tr.shape[1])))
    casca.fit(X_eeg_tr, X_imu_tr)

    Ztr, Wtr = casca.transform(X_eeg_tr, X_imu_tr)
    Zte, Wte = casca.transform(X_eeg_te, X_imu_te)

    FEAT_tr = np.hstack([Ztr, Wtr])
    FEAT_te = np.hstack([Zte, Wte])

    # ❷  embedding diagnostics (train set)
    vif   = compute_vif(FEAT_tr)
    cnum  = compute_cond_number(FEAT_tr)
    Xrec  = casca.reconstruct_x(X_eeg_tr)
    rmse  = compute_rmse(X_eeg_tr, Xrec)
    ev    = compute_explained_variance(X_eeg_tr, Xrec)

    row = dict(d_c=d_c, d_hid=d_hid,
               VIF=vif, log_cond_XTX=cnum,
               RMSE_recon=rmse, ExplVar=ev)

    # ❸  classifiers
    for name, clf in make_clfs().items():
        clf.fit(FEAT_tr, y_tr)
        y_pred = clf.predict(FEAT_te)

        row[f"{name}_F1w"] = f1_score(y_te, y_pred, average="weighted")
        row[f"{name}_F1m"] = f1_score(y_te, y_pred, average="macro")

    return row

# ================================================================
# 3)  Run over grid & collect
# ================================================================
casca_traj_results = []
for d_c in tqdm(grid_d_c):
    for d_hid in tqdm(grid_dhid):
        if d_c > d_hid:                 # skip invalid combos
            continue
        casca_traj_results.append(run_grid_point(d_c, d_hid))
        gc.collect()

casca_traj_df = pd.DataFrame(casca_traj_results)
casca_traj_df.to_csv('../experiment-results/human-player/trajectory_casca.csv', index=False)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  ➜ d_c= 1  d_hid= 2
  ➜ d_c= 1  d_hid= 4
  ➜ d_c= 1  d_hid= 6
  ➜ d_c= 1  d_hid= 8
  ➜ d_c= 1  d_hid=10
  ➜ d_c= 1  d_hid=12
  ➜ d_c= 1  d_hid=14
  ➜ d_c= 1  d_hid=16
  ➜ d_c= 1  d_hid=30
  ➜ d_c= 1  d_hid=50
  ➜ d_c= 1  d_hid=70
  ➜ d_c= 1  d_hid=90


  0%|          | 0/12 [00:00<?, ?it/s]

  ➜ d_c= 3  d_hid= 4
  ➜ d_c= 3  d_hid= 6
  ➜ d_c= 3  d_hid= 8
  ➜ d_c= 3  d_hid=10
  ➜ d_c= 3  d_hid=12
  ➜ d_c= 3  d_hid=14
  ➜ d_c= 3  d_hid=16
  ➜ d_c= 3  d_hid=30
  ➜ d_c= 3  d_hid=50
  ➜ d_c= 3  d_hid=70
  ➜ d_c= 3  d_hid=90


...

  ➜ d_c=20  d_hid=30
  ➜ d_c=20  d_hid=50
  ➜ d_c=20  d_hid=70
  ➜ d_c=20  d_hid=90


  0%|          | 0/12 [00:00<?, ?it/s]

  ➜ d_c=30  d_hid=30
  ➜ d_c=30  d_hid=50
  ➜ d_c=30  d_hid=70
  ➜ d_c=30  d_hid=90


  0%|          | 0/12 [00:00<?, ?it/s]

  ➜ d_c=40  d_hid=50
  ➜ d_c=40  d_hid=70
  ➜ d_c=40  d_hid=90


### PurePCA and PureCCA

In [186]:
lag_set = [0]               # ignore lag scan in this experiment
grid_d_hid = [2, 4, 6, 8, 10, 12, 14, 16, 30, 50, 70, 90]


def make_clfs():
    return {
        "LogReg": LogisticRegression(
            max_iter=2000,
            class_weight='balanced'
        ),
        "kNN": KNeighborsClassifier(n_neighbors=5, n_jobs=4),
        "CatBoost": CatBoostClassifier(
            iterations=200,
            verbose=False,
            task_type='CPU',
            devices='0',
            depth=6,
            learning_rate=0.05,
            loss_function='MultiClass',
            random_state=SEED
        )
    }

# helper: run one grid point
def run_grid_point(d_hid, dim_red_model_cls):
    # ❶  fit CaSCA on *train* only
    dr_model = dim_red_model_cls(d_hid)
    dr_model.fit(X_eeg_tr, X_imu_tr)

    Ztr, Wtr = dr_model.transform(X_eeg_tr, X_imu_tr)
    Zte, Wte = dr_model.transform(X_eeg_te, X_imu_te)

    FEAT_tr = np.hstack([Ztr, Wtr])
    FEAT_te = np.hstack([Zte, Wte])

    # ❷  embedding diagnostics (train set)
    vif   = compute_vif(FEAT_tr)
    cnum  = compute_cond_number(FEAT_tr)
    Xrec  = dr_model.reconstruct_x(X_eeg_tr)
    rmse  = compute_rmse(X_eeg_tr, Xrec)
    ev    = compute_explained_variance(X_eeg_tr, Xrec)

    row = dict(d_hid=d_hid,
               VIF=vif, log_cond_XTX=cnum,
               RMSE_recon=rmse, ExplVar=ev)

    # ❸  classifiers
    for name, clf in make_clfs().items():
        clf.fit(FEAT_tr, y_tr)
        y_pred = clf.predict(FEAT_te)

        row[f"{name}_F1w"] = f1_score(y_te, y_pred, average="weighted")
        row[f"{name}_F1m"] = f1_score(y_te, y_pred, average="macro")

    return row

# ================================================================
# 3)  Run over grid & collect
# ================================================================
traj_pure_results = []
for d_hid in tqdm(grid_d_hid):
    for dim_red_model_cls in (PurePCA, PureCCA):
        row = run_grid_point(d_hid, dim_red_model_cls)
        row['din_red_model'] = type(dim_red_model_cls(1)).__name__
        traj_pure_results.append(row)
    gc.collect()

traj_pure_df = pd.DataFrame(traj_pure_results)
traj_pure_df.to_csv('../experiment-results/human-player/trajectory_other.csv', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]